In [ ]:
# Часть I - генерация изначального хронометража

import gpxpy # Анализ gpx файлов
import gpxpy.gpx 
import os
import datetime
from datetime import timedelta
import speech_recognition as sr # Распознование голоса
from docx import Document # Docx документы
from docx.shared import Pt
from docx.shared import RGBColor
from pydub import AudioSegment #
from exif import Image # Парсер Расширенного формат файлов
import pytz # Часовые пояса
import functools
import math

r = sr.Recognizer() # Экземпляр распознователя голоса
d = Document() # Word-документ с хронометражом 

tz = pytz.timezone("Europe/Moscow") # Часовой пояс
gpx_dir = "tracks" # Относительный адресс папки с треком
points_dir = "points" # Относительный адресс папки с точками
path_dict = "records" # Относительный адресс непустой папки с диктофонными записями
path_photo = "photos" # Относительный адресс непустой папки с фото
result = "report.docx" # Файл для сохранения хронометража

files = os.listdir(path_dict)
files = [os.path.join(path_dict, file) for file in files]
files = [file for file in files if os.path.isfile(file)]
files.sort(key=os.path.getmtime) # Диктофонные записи

photo = os.listdir(path_photo)
photo = [os.path.join(path_photo, file) for file in photo]
photo = [file for file in photo if os.path.isfile(file)]
photo = list(map(lambda f: (f, datetime.datetime.strptime(Image(open(f, 'rb')).datetime_original, '%Y:%m:%d %H:%M:%S')), photo))
photo.sort(key=(lambda t: t[1])) #Фотографии

radial = [] #[(5,2.4), (8,1.9)] # Радиалки в формате (номер дня, длинна в км)
totalrad = 0 # Итого радиально

skip = [] # Днёвки в формате номеров дней - из километража исключаются

parts = os.listdir(gpx_dir)
parts = [(os.listdir(os.path.join(gpx_dir, part)), part) for part in parts] if "part1" in parts else [(parts, "")]
parts = [[gpxpy.parse(open(os.path.join(gpx_dir, part_name, gpx_file),'r')) for gpx_file in part] for part, part_name in parts]
def join(a, v):
    for track in v.tracks:
        a.tracks.append(track)
    for waypoint in v.waypoints:
        if not waypoint.time is None:
            a.waypoints.append(waypoint)
    return a
parts = [functools.reduce(join, part, gpxpy.gpx.GPX()) for part in parts]
    # Каждая часть это gpx из нескольких треков. Окончание одного трека - это начало другого, 
    # однако части между собой так не соединяются

points = os.listdir(points_dir)
points = [gpxpy.parse(open(os.path.join(points_dir, gpx_file),'r')) for gpx_file in points]
points = functools.reduce(join, points, gpxpy.gpx.GPX())
points = points.waypoints

    
prpoint = None # Предыдущая точка
dbpoint = None # Точка начала дня

dayn = 0 # Счётчик дней
delay = 300 # Минимальное время остановки в секундах. 
            # Если время между двумя точками превышает этот интервал - то это отмечается в хронометраже
nd_hour = 3 # В три часа ночи начинается трек нового дня
    
dist = 0 # Глобальная дистанция
ddist = 0 # Дневная дистанция
high = 0 # Глобальный набор высоты 
dhigh = 0 # Дневной набор высоты
low = 0 # Глобальный сброс высоты
dlow = 0 # Дневной сброс высоты
show_seconds = False # Отображать секунды в хронометраже?
use_audio_parser = True # Использовать распознование голоса?
clean = True # Чистить трек?

# Широту/Долготу в строку
def get_cs(coord):
    return "{0:}° {1}' {2:.2f}''".format(math.trunc(coord),math.trunc(coord*60)%60,round(coord*3600,2)%60)

# время следущей диктофонной
def dicttime():
    return datetime.datetime.fromtimestamp(os.path.getmtime(files[0]))

# Точку в строку
def get_csp(point):
    return "{0} {1}, {2} {3}".format('N' if point.latitude>=0 else 'S',get_cs(point.latitude),
                                     'E' if point.longitude>=0 else 'W',get_cs(point.longitude))
def zapis(n):
    if 10 < n%100 < 20 or 4 < n%10 < 9 or n%10 == 0:
        return "записей"
    if 1 < n%10 < 5:
        return "записи"
    return "запись"

# секунды
def second(p):
    return p.time.astimezone(tz).second

# минуты
def minute(p):
    return p.time.astimezone(tz).minute

# часы
def hour(p):
    return p.time.astimezone(tz).hour
# время простановки точки
def point_time(p):
    return "{0}:{1:02d}".format(hour(p),minute(p)) if not show_seconds else "{0}:{1:02d}:{2:02d}".format(hour(p),minute(p),second(p))

# время    
def time(tm):
    return "{0}:{1:02d}".format(tm.hour,tm.minute) if not show_seconds else "{0}:{1:02d}:{2:02d}".format(tm.hour,tm.minute,tm.second)

# Распознать аудиозапись
def getdictstr_rec(audio):
    try:
        return r.recognize_google(audio, language="Ru-ru")#recognize_wit(audio, key=wit_key)
    except sr.UnknownValueError:
        return "Запись"
    except sr.RequestError as e:
        #print(e)
        return str(e)
        #return getdictstr_rec(audio)

# Диктофонную запись в строку
def getdictstr(file):
    if use_audio_parser:
        wma = AudioSegment.from_file(os.path.abspath(file), "asf")
        wma.export("temp.wav", format="wav")
        with sr.AudioFile("temp.wav") as source:
            audio = r.record(source)
        return getdictstr_rec(audio)
    else:
        return os.path.basename(file)
        
# Печать в word
def wordprint(s, b = False, c = False, color = None):
    print(s)
    p = d.add_paragraph()
    r = p.add_run(s)
    r.bold = b
    r.italic = c
    font = r.font
    font.name = 'Times New Roman'
    font.size = Pt(12)
    font.color.rgb = color
    return p

# Альтернативная печать в word
def wordprint2(s1, s2, color = None):
    p = wordprint(s1, b = True, color = color)
    r = p.add_run(s2)
    font = r.font
    font.name = 'Times New Roman'
    font.size = Pt(12)
    font.color.rgb = color
    print(s2)
    return p

# Добавить в хронометраж диктофонные записи к данной точке
def printdict(p, f_all=True):
    if len(files) > 0:
        tm = dicttime()
        while (len(files) > 0 
               and tz.localize(tm) < p.time.astimezone(tz) 
               and (f_all or tm.hour >= nd_hour 
                    and tm.day == p.time.astimezone(tz).day-1 
                    or tm.hour < nd_hour 
                    and tm.day == p.time.astimezone(tz).day)):
            pstr = getdictstr(files[0])
            wordprint2("{0}\u00A0({1:.2f}\u00A0км/{2:.2f}\u00A0км)\u00A0"
                       .format(time(tm),ddist/1000.0,dist/1000.0),pstr)
            #print(files[0])
            files.remove(files[0])
            if len(files) > 0:
                tm = dicttime()
    return

# Добавить в хронометраж имена фото-файлов к данной точке
def printphoto(p, f_all=True):
    if len(photo) > 0:
        tm = photo[0][1]
        while (len(photo) > 0 
               and tz.localize(tm) < p.time.astimezone(tz) 
               and (f_all or tm.hour >= nd_hour 
                    and tm.day == p.time.astimezone(tz).day-1 
                    or tm.hour < nd_hour
                    and tm.day == p.time.astimezone(tz).day)):
            pstr = os.path.basename(photo[0][0])
            wordprint2("{0}\u00A0({1:.2f}\u00A0км/{2:.2f}\u00A0км)\u00A0"
                       .format(time(tm),ddist/1000.0,dist/1000.0),pstr)
            photo.remove(photo[0])
            if len(photo) > 0:
                tm = photo[0][1]
    return

# Добавить в хронометраж имена путевых точек к данной точке
def printpoints(p, f_all=True):
    if len(points) > 0:
        tm = points[0].time
        while (len(points) > 0 
               and tm < p.time.astimezone(tz) 
               and (f_all or tm.hour >= nd_hour 
                    and tm.day == p.time.astimezone(tz).day-1 
                    or tm.hour < nd_hour
                    and tm.day == p.time.astimezone(tz).day)):
            pstr = 'Точка {0} -> {1}.'.format(points[0].name, get_csp(points[0]))
            wordprint2("{0}\u00A0({1:.2f}\u00A0км/{2:.2f}\u00A0км)\u00A0"
                       .format(point_time(points[0]),ddist/1000.0,dist/1000.0),pstr)
            points.remove(points[0])
            if len(points) > 0:
                tm = points[0].time
    return

# Вернуть диктофонные записи и их число в данной точке
def dictn(p):
    count = 0
    pstr = ""
    if len(files) > 0:
        tm = dicttime()
        while len(files) > 0 and tz.localize(tm) < p.time.astimezone(tz):
            pstr += getdictstr(files[0]) + ". "
            files.remove(files[0])
            count+=1
            if len(files) > 0:
                tm = dicttime()
    return (pstr, count) #if use_audio_parser else (str(count) + " " + zapis(count) + ".", count)

# Вернуть имена файлов с фото и их число в данной точке
def photon(p):
    count = 0
    pstr = ""
    if len(photo) > 0:
        tm = photo[0][1]
        while len(photo) > 0 and tz.localize(tm) < p.time.astimezone(tz):
            pstr+= ("" if pstr == "" else " ") + os.path.basename(photo[0][0])
            count+=1
            photo.remove(photo[0])
            if len(photo) > 0:
                tm = photo[0][1]
    return (pstr + ".", count)

# Вернуть имена путевых точек и их число в данной точке
def pointn(p):
    count = 0
    pstr = ""
    if len(points) > 0:
        tm = points[0].time
        while len(points) > 0 and tm < p.time.astimezone(tz):
            pstr += 'Точка {0} -> {1}.'.format(points[0].name, get_csp(points[0]))
            count += 1
            points.remove(points[0])
            if len(points) > 0:
                tm = points[0].time
    return (pstr + ".", count)

# Печать путевых точек, фото и диктофона к данной точке
def printdnf(p, f_all=True):
    printpoints(p, f_all = f_all)
    printdict(p, f_all = f_all)
    printphoto(p, f_all = f_all)
    return

# dictn + photon + pointn
def dnfnpn(p):
    (t0, t1) = dictn(p)
    (t2, t3) = photon(p)
    (t4, t5) = pointn(p)
    return (t0, t1, t2, t3, t4, t5)

for gpx in parts:
    prpoint = None
    for track in gpx.tracks:
        for segment in track.segments:
            
            # Чистим трек от точек проставленных на скорости <= 2 км/ч
            if clean:
                pns = list()
                pn = 0
                for p1, p2 in zip(segment.points[:-1], segment.points[1:]):
                    if (p1.distance_3d(p2)/p1.time_difference(p2)*3.6) <= 2.0:
                        pns.append(pn)
                    pn+=1
                for p in pns[::-1]:
                    segment.remove_point(p)
            
            # Обработка дней похода
            for point in segment.points:
                if prpoint == None:
                    prpoint = point
                if ((dbpoint == None) 
                    or (point.time.astimezone(tz).day != dbpoint.time.astimezone(tz).day 
                        and hour(point) >= nd_hour)):
                    delta = (point.time.date() - prpoint.time.date()).days
                    dayn += 1 if delta <= 1 else delta
                    if dbpoint != None:
                        wordprint2("{0}\u00A0({1:.2f}\u00A0км/{2:.2f}\u00A0км)\u00A0"
                                   .format(point_time(prpoint),ddist/1000.0,dist/1000.0), "Окончание ходового дня.", color = RGBColor(0, 176, 80))
                        printdnf(point, f_all = False)
                        wordprint("Пройденное расстояние: {0:.2f} км".format(ddist/1000.0), c=True)
                        if len(radial) > 0 and radial[0][0] == dayn - 1:
                            wordprint("из них радиально: {0:.2f} км".format(radial[0][1]), c=True)
                            totalrad += radial[0][1]
                            radial.remove(radial[0])
                        wordprint("Набор высоты: {0:.1f} м".format(dhigh), c=True)
                        wordprint("Сброс высоты: {0:.1f} м".format(dlow), c=True)
                        wordprint("Высота ночёвки: {0:.1f} м".format(point.elevation), c=True)
                        wordprint("Координаты точки ночёвки: {0}".format(get_csp(point)), c=True)
                        wordprint("")
                    else:
                        printdnf(point)
                        wordprint("")
                    wordprint("День {0}. {1}".format(dayn,point.time.astimezone(tz).date()),b=True,c=True)
                    wordprint("МС{0} \u2014 МС{1}".format(dayn-1,dayn),c=True)
                    if dayn in skip:
                        wordprint("Днёвка.")
                    dbpoint = point
                    ddist = 0
                    dhigh = 0
                    dlow = 0
                    printdnf(point)
                    wordprint2("{0}\u00A0({1:.2f}\u00A0км/{2:.2f}\u00A0км)\u00A0".format
                        (point_time(point),ddist/1000.0,dist/1000.0), "Начало ходового дня.", color = RGBColor(0, 176, 80))
                else:
                    if not dayn in skip:
                        dist += prpoint.distance_3d(point)
                        if point.elevation-prpoint.elevation > 0:
                            high += point.elevation-prpoint.elevation
                        else:
                            low += prpoint.elevation-point.elevation
                    ddist += point.distance_3d(prpoint)
                    if point.elevation-prpoint.elevation > 0:
                            dhigh += point.elevation-prpoint.elevation
                    else:
                            dlow += prpoint.elevation-point.elevation
                    if point.time_difference(prpoint) > delay:
                        (dicts, x, photos, y, waypoints, z) = dnfnpn(point)
                        wordprint2("{0}-{1}\u00A0({2:.2f}\u00A0км/{3:.2f}\u00A0км)\u00A0"
                                   .format(point_time(prpoint),point_time(point),ddist/1000.0,dist/1000.0),
                                           ("" if z == 0 else waypoints)+("" if x == 0 else dicts)+("" if y == 0 else photos))
                    else:
                        printdnf(point)
                prpoint = point
  
 # Последний день с итогами
if dbpoint != None:
    wordprint2("{0}\u00A0({1:.2f}\u00A0км/{2:.2f}\u00A0км)\u00A0"
               .format(point_time(point),ddist/1000.0,dist/1000.0), "Окончание ходового дня.", color = RGBColor(0, 176, 80))
    wordprint("Пройденное расстояние: {0:.2f} км".format(ddist/1000.0), c=True)
    if len(radial) > 0 and radial[0][0] == dayn:
        wordprint("из них радиально: {0:.2f} км".format(radial[0][1]), c=True)
        totalrad += radial[0][1]
        radial.remove(radial[0])
    wordprint("Набор высоты: {0:.1f} м".format(dhigh), c=True)
    wordprint("Сброс высоты: {0:.1f} м".format(dlow), c=True)
    wordprint("Высота ночёвки: {0:.1f} м".format(point.elevation), c=True)
    wordprint("Координаты точки ночёвки: {0}".format(get_csp(point)), c=True)
    wordprint("")
    wordprint("\nИтого:", c=True)
    wordprint("    Дней: {0}".format(dayn), c=True)
    wordprint("    Пройденное расстояние: {0:.2f} км".format(dist/1000.0), c=True)
    wordprint("    Из них радиально: {0:.2f} км".format(totalrad), c=True)
    wordprint("    Набор высоты: {0:.1f} м".format(high), c=True)
    wordprint("    Сброс высоты: {0:.1f} м".format(low), c=True)

# wordprint("")
# for waypoint in points.waypoints:
#    wordprint('{0}.{1} ({2}) Точка {2} -> ({3},{4})'.format(waypoint.time.day, waypoint.time.month, point_time(waypoint), waypoint.name, waypoint.latitude, waypoint.longitude))

# Печатаем результат в файл
d.save(result)

In [ ]:
# Часть II - вставка походных фотографий на место имён файлов.
# Важно! Относительный порядок фото должен быть сохранён, при этом можно удалить лишние фотографии.

import os
import datetime
from docx import Document # Docx документы
from docx.shared import Pt
from exif import Image # Парсер Расширенного формат файлов
import pytz # Часовые пояса
import functools

path_photo = "photos" # Относительный адресс непустой папки с фото
result = "report.docx" # Файл хронометража
result_p = "report_pics.docx" # Результат с картинками

photo = os.listdir(path_photo)
photo = [os.path.join(path_photo, file) for file in photo]
photo = [file for file in photo if os.path.isfile(file)]
photo = list(map(lambda f: (f, datetime.datetime.strptime(Image(open(f, 'rb')).datetime_original, '%Y:%m:%d %H:%M:%S')), photo))
photo.sort(key=(lambda t: t[1])) #Фотографии

d = Document(result)
dp = Document()

for p in d.paragraphs:
    new_p = dp.add_paragraph()
    new_p.style = p.style
    t = p.text
    found_photo = []
    n = 0
    while len(photo) > 0:
        ph_n = os.path.basename(photo[0][0])
        tmp_n = t.find(ph_n, n)
        if tmp_n == -1:
            tmp_n = t.find("JPG", n)
            if tmp_n == -1:
                tmp_n = t.find("jpg", n)
                if tmp_n == -1:
                    break
            photo.remove(photo[0])
            continue
        found_photo.append((photo[0][0], tmp_n, len(ph_n)))
        n = tmp_n+len(ph_n)
        photo.remove(photo[0])

    run_counter = 0
    for r in p.runs:
        new_r = new_p.add_run()
        new_r.style = r.style
        new_r.font.name = r.font.name
        new_r.font.size = r.font.size
        new_r.font.color.rgb = r.font.color.rgb
        new_r.bold = r.bold
        new_r.italic = r.italic
        new_r.underline = r.underline
        rst = run_counter
        run_counter += len(r.text)
        red = run_counter
        retry = True
        new_r.text = r.text
        while retry:
            if len(found_photo) == 0:
                retry = False
                continue
            ph, st, ln = found_photo[0]
            if rst >= st and red <= st + ln:
                if red == st + ln:
                    dp.add_picture(ph)
                    new_r.text = new_r.text[:st-rst]
                    rst+=ln+1
                    found_photo.remove(found_photo[0])
                continue
            
            if red <= st:
                retry = False
                continue
            
            if rst < st and red > st + ln:
                new_r.text = new_r.text[:st-rst] + new_r.text[st-rst+ln+1:]
                dp.add_picture(ph)
                rst+=ln+1
                found_photo.remove(found_photo[0])
            elif rst >= st:
                new_r.text = new_r.text[st-rst+ln+1:]
                dp.add_picture(ph)
                rst+=ln+1
                found_photo.remove(found_photo[0])
            elif red < st + ln:
                retry = False
                new_r.text = new_r.text[:st-rst]
        
            
dp.save(result_p)

In [ ]:
# Часть III - подсчёт статистики.
# На вход подаётся раскрашенный документ.
# Зелёным цветом выделяются сообщения о начале и окончании ходового дня.
# Эти сообщения вставляются автоматически на первом этапе.
# Синим - сообщения о смене покрытия, а также начале/окончании радиальных участков.
# Важно выделять сообщения вместе с меткой времени/километража, а также заканчивать каждое сообщение точкой.
# Необходимо также указать самое первое покрытие сразу после начала первого ходового дня.
# Цвета в формате RGB вычисляются по формуле B-G-R>=50 (для синего цвета). 

# Форматы сообщений: 
# 7:55 (0.00 км/303.80 км) Начало ходового дня. 
#   - изменения не допускаются.
# 17:28 (88.48 км/392.28 км) Окончание ходового дня. 
#   - изменения не допускаются.
# 8:36 (0.62 км/304.42 км) Начало радиального участка. 
#   - изменения не допускаются.
# 8:44 (2.93 км/306.73 км) Окончание радиального участка. 
#   - изменения не допускаются.
# 8:36 (0.62 км/304.42 км) Покрытие – асфальт хорошего качества. 
#   - вместо фразы асфальт хорошего качества необходимо вставить нужное покрытие.
#   - внимание! есть поддержка длинного тире из ворда.
# 8:11-8:17 (0.79 км/449.69 км) Покрытие – грунт разбитый/грунт с ТП (30/70).
#   - формат смешанных покрытий указывается через "/", в скобках указывается соотношение

import os
import datetime
from docx import Document # Docx документы
from docx.shared import Pt
from exif import Image # Парсер Расширенного формат файлов
import pytz # Часовые пояса
import functools
import re

result_p = "report_pics.docx" # Файл хронометража
result_t = "report_tabs.docx" # Результат с таблицами

d_source = Document(result_p)
d = Document()
days = []
daylen = ""
radial = False
surfs = ""
message = r'\d{1,2}:\d{2}(?:-\d{1,2}:\d{2})?[\xA0 ]\((\d+\.\d{2})[\xA0 ]км/\d+\.\d{2}[\xA0 ]км\)[\xA0 ]((?:\w|-|–|\s|\d|\(|\)|/)+)\.'
surf = r'Покрытие [-–] ((?:\w|\s)+)(?:/((?:\w|\s)+) \((\d{1,2})/(\d{1,2})\))?'
current_surf = ([], [], False)
dayn = 1 # номер стартового дня

# разделение покрытий с использованием точек в качестве разделителей.
def split_surfs(surfs):
    res = []
    p0 = 0
    p1 = surfs.find('.', p0) + 1
    while p1 != 0:
        p2 = surfs.find('.', p1) + 1
        p3 = surfs.find('.', p2) + 1
        res.append(surfs[p0:p3])
        p0 = p3
        p1 = surfs.find('.', p0) + 1
    return res

# Выделение типа и точки начала покрытия, радиальные участки также считаются как смена покрытия
def parse_surf(s):
    global current_surf
    global radial
    try:
        #print(s)
        ps = re.search(message, s.strip())
        if ps[2] == "Начало радиального участка":
            radial = True
            current_surf = (current_surf[0], current_surf[1], radial)
        elif ps[2] == "Окончание радиального участка":
            radial = False
            current_surf = (current_surf[0], current_surf[1], radial)
        else:
            sf = re.search(surf, ps[2].strip())
            if sf[2] is None:
                current_surf = ([sf[1]], [100.0], radial)
            else:
                current_surf = ([sf[1], sf[2]], list(map(float, [sf[3], sf[4]])), radial)
        res = (float(ps[1]), current_surf)
    except:
        print("!!!" + s)
        res = (0.0, "ошибка в " + e)
    return res

# Разделение дня на учачстки по покрытиям
def split_day(day):
    #print(day)
    surfs = split_surfs(day[1])
    #print(surfs)
    surfs = list(map(parse_surf, surfs))
    #print()
    return (day[0], surfs)

# Объединение однотипных покрытий за день
def stack_day(day):
    d = dict()
    global current_surf
    current_dist = 0
    for surf in day[1]:
        if current_surf != ([], [], ""):
            for (sf, sz) in zip(current_surf[0], current_surf[1]):
                d[(sf, current_surf[2])] = (0.0 if d.get((sf, current_surf[2])) is None \
                                       else d[(sf, current_surf[2])]) + (surf[0] - current_dist)/100.0*sz
        current_dist, current_surf = surf
    for (sf, sz) in zip(current_surf[0], current_surf[1]):
        d[(sf, current_surf[2])] = (0.0 if d.get((sf, current_surf[2])) is None \
                                       else d[(sf, current_surf[2])]) + (day[0] - current_dist)/100.0*sz
    d = dict(sorted(d.items(), key=lambda x: x[0]))
    return (day[0], d)

# Печать в ворд
def wordprint(s, b = False, c = False, color = None):
    print(s)
    p = d.add_paragraph()
    r = p.add_run(s)
    r.bold = b
    r.italic = c
    font = r.font
    font.name = 'Times New Roman'
    font.size = Pt(12)
    font.color.rgb = color
    return p

# Печать статистики за день в ворд
def print_day(d, total = False):
    if total:
        wordprint("Итого".format(dayn), b=True)
    else:
        wordprint("День {0}".format(dayn), b=True)
    wordprint("Пройденное расстояние: {0:.2f} км".format(d[0]), c=True)
    surfs = []
    rad = 0.0
    for surf, dist in d[1].items():
        if dist != 0.0:
            surfs.append("{0}{1} – {2:.2f} км".format(surf[0], " (радиально)" if surf[1] else "", dist))
        if surf[1]:
            rad += dist
    
    wordprint("из них" + ("" if rad == 0.0 else " радиально {0:.2f} км".format(rad)), c=True)
    for surf in surfs:
        wordprint(surf, c=True)
    
    wordprint("", c=True)
    return


def add_day(d1, d2):
    d = d1[1]
    for surf, dist in d2[1].items():
        if d.get(surf) is None:
            d[surf] = dist
        else:
            d[surf] += dist
    d = dict(sorted(d.items(), key=lambda x: x[0]))
    return (d1[0] + d2[0], d)

for p in d_source.paragraphs:
    for run in p.runs:
        (r, g, b) = (0, 0, 0) if run.font.color.rgb is None else run.font.color.rgb
        if b == r == g == 0 or b - r - g >= 50:
            if daylen != "":
                try:
                    dl = re.search(message, daylen.strip())
                    if dl[2] == "Начало ходового дня":
                        surfs = ""
                    elif dl[2] == "Окончание ходового дня":
                        days.append((float(dl[1]), surfs))
                except:
                    print("!!" + daylen)
                daylen = ""
            if b - r - g >= 50:
                surfs += run.text
        if g - b - r >= 50:
            daylen += run.text
print(days)
days = list(map(split_day, days))
currnet_surf = ([], [], False)
days = list(map(stack_day, days))
total = (0.0, dict())
for day in days:
    print_day(day)
    total = add_day(total, day)
    dayn += 1
print_day(total, total=True)
            
d.save(result_t)